In [1]:
!pip install pymongo
!pip install langchain_community
!pip install unsloth
!pip install langgraph
!pip install bert-score
!pip install unbabel-comet
!pip install tqdm
!pip install openpyxl
!pip install openai


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
  Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (679 bytes)
Using cached protobuf-3.20.3-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 4.25.6
    Uninstalling protobuf-4.25.6:
      Successfully uninstalled protobuf-4.25.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unbabel-comet 2.2.4 requires protobuf<5.0.0,>=4.24.4, but you have protobuf 3.20.3 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip i

In [2]:
!pip install peft


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
import langchain
import langgraph
from langgraph.graph import StateGraph, START, END, add_messages
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, HumanMessage, AIMessage
import torch
from bert_score import score as bert_score
from comet import download_model, load_from_checkpoint
import os
from typing import TypedDict, List
from huggingface_hub import login
from pymongo import MongoClient
from pymongo.database import Database
from pymongo.collection import Collection
import requests
import json
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import AzureChatOpenAI
import pandas as pd
import numpy as np
from transformers import XLMRobertaTokenizer, XLMRobertaModel
from unsloth import FastLanguageModel
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

PyTorch version 2.6.0 available.


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


comet_ml is installed but the Comet API Key is not configured. Please set the `COMET_API_KEY` environment variable to enable Comet logging. Check out the documentation for other ways of configuring it: https://www.comet.com/docs/v2/guides/experiment-management/configure-sdk/#set-the-api-key


In [4]:
import os
from dotenv import load_dotenv
from huggingface_hub import login

# .env 파일 로드
load_dotenv()

# 환경 변수 가져오기 (KeyError 방지하려면 .get()을 추천)
HUGGINGFACE_TOKEN = os.environ["HUGGING_FACE_TOKEN"]  # 환경 변수 없으면 KeyError 발생

# 로그인
login(token=HUGGINGFACE_TOKEN)


In [5]:
from datasets import load_dataset
from peft import PeftModel
from transformers import AutoModelForCausalLM
import pandas as pd

BASE_MODEL_NAME = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
PEFT_MODEL_NAME = "haeun0420/gorani-1b-cos4"

# MODEL_PATH = "/workspace/haeun0420/gorani-1b-cos4"  

# ✅ 기본 모델 로드
base_model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_NAME)

try:
    # ✅ PEFT 모델 로드 (한 번만 실행)
    model = PeftModel.from_pretrained(base_model, PEFT_MODEL_NAME)
    print("✅ PEFT 모델 로드 성공")
except Exception as e:
    print(f"❌ PEFT 모델 로드 실패: {e}")

# ✅ 모델 및 데이터 경로 설정
OUTPUT_CSV_PATH = "/workspace/test_results.csv"  # 결과 저장 경로


`low_cpu_mem_usage` was None, now default to True since model is quantized.


✅ PEFT 모델 로드 성공


In [6]:
from datasets import load_dataset
import pandas as pd

ds = load_dataset("aripos1/gorani_dataset")

# 'train' 데이터셋을 pandas DataFrame으로 변환
df_test = ds["test"].to_pandas()  # 'test' split이 있는지 확인 필요

# ✅ 데이터 확인
print(f"✅ 테스트 데이터 로드 완료 ({len(df_test)} 개 샘플)")
print(df_test.head())  # 데이터 구조 확인


✅ 테스트 데이터 로드 완료 (5416 개 샘플)
                                         instruction  \
0  Translate the English into target language. Re...   
1  Translate the Korean into target language. Ref...   
2  Translate the English into target language. Re...   
3  Translate the Korean into target language. Ref...   
4  Translate the Korean into target language. Ref...   

                                               input  \
0  I decided to take a walk with my friends at Gu...   
1                                 원적산의 경치는 정말 아름다워요.   
2  The movie filmed at the Iksan Prison Film Set ...   
3  여러분, 한국의 매력적인 음식 문화 속에서 '오징어두루치기'를 통해 신선한 해산물과...   
4  고석정 꽃밭은 아름다운 자연경관과 다채로운 꽃들이 어우러져 여행자 여러분을 환상적인...   

                                              output target_language  \
0                       구봉산 근린공원에서 친구들과 산책을 하기로 했어요.              KO   
1  The scenery of Wonjeok Mountain is truly beaut...             ENG   
2                  익산 교도소 세트장에서 촬영한 영화가 큰 인기를 끌고 있다.              KO   
3  皆さん、韓国の

In [7]:
max_seq_length = 512 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

In [8]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=PEFT_MODEL_NAME,
    max_seq_length = max_seq_length,
    dtype = None,
    load_in_4bit = True,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth 2025.2.5: Fast Llama patching. Transformers: 4.48.3.
   \\   /|    GPU: NVIDIA RTX 2000 Ada Generation. Max memory: 15.699 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.9. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.2.5 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048, padding_idx=128004)
        (layers): ModuleList(
          (0): LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Identity()
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=32, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=32, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear

In [9]:
# ✅ Hugging Face Pipeline 생성
hf_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False,
 )
print("✅ Pipeline 생성 완료!")


Device set to use cuda:0
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'GraniteForCausalLM', 'GraniteMoeForCausalLM', 'Jam

✅ Pipeline 생성 완료!


In [10]:
import os
import json
import torch
import pandas as pd
from datasets import load_dataset
from transformers import pipeline, AutoTokenizer
from peft import PeftModel
from unsloth import FastLanguageModel

def apply_prompt_template(instruction, input, glossary):
    alpaca_prompt = (
        "Below is an instruction that describes a task, paired with an input that provides further context. "
        "Write a response that appropriately completes the request.\n"
        "\n\n### Instruction:\n"
        f"{instruction}"
        "\n\n### Input:\n"
        f"{input}"
        "\n\n### Glossary:\n"
        f"{glossary}"   
        "\n\n### Response:"
    )
    
    return alpaca_prompt

In [11]:
import json

def generate_translation(model, tokenizer, instruction, input_, glossary):
    """
    모델을 사용하여 입력 텍스트를 번역하는 함수.
    """
    # ✅ Glossary가 dict라면 JSON-like 문자열로 변환
    if isinstance(glossary, dict):
        glossary = json.dumps(glossary, ensure_ascii=False)

    # ✅ 프롬프트 생성
    prompt = apply_prompt_template(instruction, input_, glossary)

    # ✅ 모델 토크나이징
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # GPU 사용

    # ✅ 모델 추론 수행
    with torch.no_grad():
        output_ids = model.generate(
            **inputs, 
            max_length=512,  # 생성할 문장의 최대 길이
            temperature=0.7,  # 창의성 조절 (0.7~1.0 추천)
            top_p=0.9,  # 확률 질량을 기반으로 토큰 샘플링
            repetition_penalty=1.1,  # 반복 방지
            do_sample=True  # 샘플링 사용
        )

    # ✅ 토큰을 텍스트로 변환
    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # ✅ "Response:" 이후의 번역된 텍스트만 추출
    if "### Response:" in generated_text:
        generated_text = generated_text.split("### Response:")[-1].strip()

    return generated_text


In [ ]:
# ✅ 결과 저장 리스트
results = []

# ✅ 데이터셋 순회하며 번역 수행
for idx, row in df_test.iterrows():
    instruction = row["instruction"]
    input_ = row["input"]  # ✅ 데이터프레임 컬럼명 유지
    expected_output = row["output"]
    target_language = row["target_language"]
    glossary_text = row["metadata"]  # glossary 정보 (메타데이터에 포함됨)

    # ✅ 모델을 사용하여 번역 생성 (glossary 추가!)
    generated_output = generate_translation(model, tokenizer, instruction, input_, glossary_text)

    # ✅ 출력 형식 맞춰서 표시
    print("\n" + "="*50)
    print(f"🔹 샘플 {idx+1}/{len(df_test)}")
    print(f"🔹 입력: {input_}")  # ✅ 변수명 변경
    print(f"✅ 예상 번역: {expected_output}")
    print(f"✅ 모델 번역: {generated_output}")
    print(f"✅ Target Language: {target_language}")
    print(f"✅ Glossary: {glossary_text}")
    print("="*50 + "\n")

  
# ✅ 결과 저장
    results.append({
        "input": input_,  # ✅ 원래 컬럼명 유지
        "expected_output": expected_output,
        "generated_output": generated_output,
        "target_language": target_language,
        "glossary": glossary_text
    })

    # 진행 상황 출력 (10개 단위)
    if (idx + 1) % 10 == 0:
        print(f"\n✅ {idx+1}/{len(df_test)} 개 처리 완료...\n")

# ✅ 결과 DataFrame 변환
df_results = pd.DataFrame(results)

# ✅ 결과 CSV 파일로 저장
df_results.to_csv(OUTPUT_CSV_PATH, index=False, encoding="utf-8-sig")

print(f"\n✅ 전체 추론 완료! 결과 저장 경로: {OUTPUT_CSV_PATH}")



🔹 샘플 1/5416
🔹 입력: I decided to take a walk with my friends at Gubongsan Neighborhood Park.
✅ 예상 번역: 구봉산 근린공원에서 친구들과 산책을 하기로 했어요.
✅ 모델 번역: 구봉산 근린공원에서 친구들과 함께 산책을 하기로 했어요.
✅ Target Language: KO
✅ Glossary: {'ENG': 'Gubongsan Neighborhood Park', 'JPN': '九峰山近隣公園', 'KO': '구봉산 근린공원'}


🔹 샘플 2/5416
🔹 입력: 원적산의 경치는 정말 아름다워요.
✅ 예상 번역: The scenery of Wonjeok Mountain is truly beautiful.
✅ 모델 번역: The scenery of Wonjeok Mountain is truly beautiful.
✅ Target Language: ENG
✅ Glossary: {'ENG': 'Wonjeok Mountain', 'JPN': '原積算', 'KO': '원적산'}


🔹 샘플 3/5416
🔹 입력: The movie filmed at the Iksan Prison Film Set is gaining a lot of popularity.
✅ 예상 번역: 익산 교도소 세트장에서 촬영한 영화가 큰 인기를 끌고 있다.
✅ 모델 번역: 익산 교도소 세트장을 사용한 영화는 많은 인기를 끌고 있다.
✅ Target Language: KO
✅ Glossary: {'ENG': 'Iksan Prison Film Set', 'JPN': '益山刑務所セット場', 'KO': '익산 교도소 세트장'}


🔹 샘플 4/5416
🔹 입력: 여러분, 한국의 매력적인 음식 문화 속에서 '오징어두루치기'를 통해 신선한 해산물과 매콤한 양념이 어우러진 특별한 미식을 경험해 보시기 바랍니다.
✅ 예상 번역: 皆さん、韓国の魅力的な食文化の中で「イカの野菜炒め」を通じて新鮮な海鮮と辛い調味料が調和した特別な美食を体験してみてください。
✅ 모